# Modelagem dos Dados

### *Descrição do problema*
Você é um funcionário da OMS que deve avaliar os níveis de contaminação de um vírus em um determinado país. As pessoas dentro de uma sociedade podem estar conectadas de alguma maneira (familia, amizade ou trabalho) e cada pessoa possui um conjunto de atributos.
Este vírus afeta esta sociedade como descrito a seguir:
- a taxa de contaminação varia de pessoa para pessoa;
- a taxa de contaminação de uma pessoa A para B é diferente de B para A e depende das características de ambas as pessoas (A e B);
- a contaminação só passa através de indivíduos conectados;
- não existe cura para essa doença;


### *O desafio*
Foram coletados os dados de contaminação (ou seja, as taxas de contaminação) para metade desta sociedade. Neste problema, você deverá estimar a taxa para o restante dessa sociedade e decidir políticas de saúde com base nos resultados obtidos.
Observação: Para determinar as taxas de contaminação, devem ser levados em consideração tanto as características dos infectados quanto dos infectantes.

## Importando bibliotecas

In [1]:
import math
import warnings

import numpy as np
import pandas as pd
from sklearn import linear_model, metrics, svm
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.impute import KNNImputer
from sklearn.linear_model import (LinearRegression, LogisticRegression, Ridge)
from sklearn.metrics import r2_score
from sklearn.model_selection import (GroupKFold, KFold, ShuffleSplit,
                                     StratifiedKFold, cross_validate,
                                     train_test_split)
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from tqdm import tqdm

from script_impute.DataImputation import DataImputation

In [2]:
# ignorando avisos
warnings.filterwarnings('ignore')

## Carregando os dados dos indivíduos e conexões

In [13]:
df_individuos = pd.read_csv("../data/individuos_espec.csv", sep=';')
df_conexoes = pd.read_csv("../data/conexoes_espec.csv", sep=';')

## Visualizando os dados dos indivíduos

In [10]:
print("Shape:", df_individuos.shape)
df_individuos.head()

Shape: (1000000, 9)


,name,idade,estado_civil,qt_filhos,estuda,trabalha,pratica_esportes,transporte_mais_utilizado,IMC
0,1,44.0,divorciado,1.0,1.0,0.0,1.0,publico,22.200956
1,2,24.0,casado,0.0,0.0,0.0,1.0,publico,25.378720
2,3,35.0,solteiro,1.0,0.0,0.0,1.0,particular,19.952393
3,4,50.0,casado,1.0,1.0,1.0,0.0,publico,26.732053
4,5,30.0,solteiro,2.0,1.0,0.0,1.0,publico,15.295668


## Info dos dados dos indivíduos

In [11]:
df_individuos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 9 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   name                       1000000 non-null  int64  
 1   idade                      904063 non-null   float64
 2   estado_civil               949927 non-null   object 
 3   qt_filhos                  971133 non-null   float64
 4   estuda                     959870 non-null   float64
 5   trabalha                   993647 non-null   float64
 6   pratica_esportes           850876 non-null   float64
 7   transporte_mais_utilizado  956967 non-null   object 
 8   IMC                        886130 non-null   float64
dtypes: float64(6), int64(1), object(2)
memory usage: 68.7+ MB


## Visualizando valores ausentes nos dados dos indivíduos

In [12]:
df_individuos.isna().sum()

name                              0
idade                         95937
estado_civil                  50073
qt_filhos                     28867
estuda                        40130
trabalha                       6353
pratica_esportes             149124
transporte_mais_utilizado     43033
IMC                          113870
dtype: int64

## Visualizando os dados das conexoes dos indivíduos

In [13]:
print("Shape:", df_conexoes.shape)
df_conexoes.head()

Shape: (999999, 5)


,V1,V2,grau,proximidade,prob_V1_V2
0,1,2,trabalho,visita_frequente,0.589462
1,1,3,trabalho,visita_rara,0.708465
2,2,4,trabalho,visita_casual,NaN
3,2,5,trabalho,visita_rara,0.638842
4,3,6,amigos,mora_junto,NaN


## Info dos dados das conexões dos indivíduos

In [14]:
df_conexoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999999 entries, 0 to 999998
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   V1           999999 non-null  int64  
 1   V2           999999 non-null  int64  
 2   grau         999999 non-null  object 
 3   proximidade  999999 non-null  object 
 4   prob_V1_V2   499999 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 38.1+ MB


## Visualizando valores ausentes nos dados de conexões dos indivíduos

In [15]:
df_conexoes.isna().sum()

V1                  0
V2                  0
grau                0
proximidade         0
prob_V1_V2     500000
dtype: int64

### Obersavções:

- __Dados dos indivíduos__: 
Conforme visualizado acima, os dados dos indivíduos possuem valores ausentes em todas as colunas. Esses dados serão preenchidos com o método [KNNImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html) do scikit-learn, este método faz a imputação para completar valores ausentes usando `k-Nearest Neighbors`.

- __Dados de conexões dos indivíduos__: 
Os dados de conexões dos indivíduos possuem, conforme esperado, valores ausentes na coluna `prob_V1_V2`. Esses dados ausentes serão preenchidos com as estimativas retornadas do modelo utilizado.

## Imputação de valores ausentes nos dados dos indivíduos
### Método de imputação [KNNImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html) 

Em um primeiro momento, todo processo foi realizado no notebook porém para torná-lo mais enxuto, optei por levar o processo para um script separado, que também pode ser encontrado neste repositório. Métodos mais simples poderiam ter sido escolhidos, por exemplo, ao analisar as medidas de tendências centrais dos dados, a mediana poderia ter sido escolhida. Entretanto, por já ter estudado, em cursos anteriores, este método de imputação, optei pelo mesmo. É válido ressaltar que a aplicação do presente método demandou pesquisas em documentações e notebooks utilizados anteriormente.

Para a diminuição do tempo de processamento, é possível acessar os dados, já imputados, rodando a célula a seguir:

In [19]:
new_df_individuos = pd.read_csv('../data/new_individuos_espec.csv')

In [24]:
new_df_individuos.head()

,name,idade,estado_civil,qt_filhos,estuda,trabalha,pratica_esportes,transporte_mais_utilizado,IMC
0,260368,54,solteiro,0,0,1,1,particular,23.371572
1,145753,25,solteiro,0,1,1,0,publico,15.039098
2,40429,16,solteiro,0,0,1,0,publico,18.390805
3,801085,20,divorciado,0,0,1,0,publico,18.040247
4,195257,23,solteiro,0,0,1,0,particular,24.664055


Caso opte por refazer o processamento, rode célula seguinte:

In [ ]:
df_imputed = DataImputation(df_individuos)
df_imputed.impute()
new_df_individuos = df_imputed.get_data()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 86/86 [06:43<00:00,  4.69s/it]


In [26]:
# salvando os dados processados em um novo .csv

# new_df_individuos.to_csv('../data/new_individuos_espec.csv')

Visualizo novamente a existência de dados ausentes nos dados do indivíduos e conforme esperado, pós processamento, os dados foram imputados.

In [30]:
new_df_individuos.isna().sum()

name                         0
idade                        0
estado_civil                 0
qt_filhos                    0
estuda                       0
trabalha                     0
pratica_esportes             0
transporte_mais_utilizado    0
IMC                          0
dtype: int64

## Preparação dos dados

#### __Dados dos indivíduos__:
- Nessa etapa, já com os dados dos indivíduos imputados, seguirei com as etapas de normalização dos dados numéricos e encode dos dados categóricos.

In [26]:
df_ind = new_df_individuos.copy()
df_con = df_conexoes.copy()

In [32]:
# normalização dos dados dos individuos

norm_params = {}
for col in df_ind.columns:
    if col != 'name':
        if df_ind[col].dtype != 'object':
            maxim = df_ind[col].max()
            minim = df_ind[col].min()
            norm_params[f'max_{col}'] = maxim
            norm_params[f'min_{col}'] = minim
            df_ind[col] = (df_ind[col] - minim)/(maxim - minim)

In [33]:
# encode dos dados dos individuos

for col in df_ind.columns:
    if col != 'name':
        if df_ind[col].dtype == 'object':
            encoding = pd.get_dummies(df_ind[col], prefix=col, prefix_sep='-')
            df_ind = df_ind.drop(col, axis=1)
            df_ind = df_ind.join(encoding)

#### __Dados das conexões dos indivíduos__:
- Encode dos dados categóricos presentes nas conexões.

In [34]:
# encode dos dados de concexões dos individuos

for col in df_con.columns:
    if col != 'name':
        if df_con[col].dtype == 'object':
            encoding = pd.get_dummies(df_con[col], prefix=col, prefix_sep='-')
            df_con = df_con.drop(col, axis=1)
            df_con = df_con.join(encoding)

## Merge dos dados de indivíduos e conexões processados

In [16]:
def create_df(df_ind, df_con):
    df_merged = pd.merge(df_ind.rename(columns=lambda x: x + '_V2'), df_con, how='right',
                         left_on='name_V2',  right_on='V2')
    df_merged = pd.merge(df_ind.rename(columns=lambda x: x + '_V1'), df_merged, how='right',
                         left_on='name_V1', right_on='V1')
    df_merged = df_merged.drop(['V1', 'V2'], axis=1)
    return df_merged

In [36]:
df_merged = create_df(df_ind, df_con)

In [37]:
df_merged.head()

,name_V1,idade_V1,qt_filhos_V1,estuda_V1,trabalha_V1,pratica_esportes_V1,IMC_V1,estado_civil-casado_V1,estado_civil-divorciado_V1,estado_civil-solteiro_V1,...,transporte_mais_utilizado-publico_V2,transporte_mais_utilizado-taxi_V2,prob_V1_V2,grau-amigos,grau-familia,grau-trabalho,proximidade-mora_junto,proximidade-visita_casual,proximidade-visita_frequente,proximidade-visita_rara
0,1,0.354839,0.111111,1.0,0.0,1.0,0.205777,0,1,0,...,1,0,0.589462,0,0,1,0,0,1,0
1,1,0.354839,0.111111,1.0,0.0,1.0,0.205777,0,1,0,...,0,0,0.708465,0,0,1,0,0,0,1
2,2,0.193548,0.000000,0.0,0.0,1.0,0.243471,1,0,0,...,1,0,NaN,0,0,1,0,1,0,0
3,2,0.193548,0.000000,0.0,0.0,1.0,0.243471,1,0,0,...,1,0,0.638842,0,0,1,0,0,0,1
4,3,0.282258,0.111111,0.0,0.0,1.0,0.179106,0,0,1,...,1,0,NaN,1,0,0,1,0,0,0


## Imputação dos dados de taxa de contaminação

### Imputação utilizando regressão pelo k vizinhos mais próximos através do algoritmo [KNeighborsRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html) 

In [3]:
# redução de dimensionalidade com PCA

cols_to_drop = ['name_V1', 'name_V2', 'prob_V1_V2']

def pca_transform(df, n_components=0.9):
    scaler = MinMaxScaler()
    pca = PCA(n_components, random_state=42)
    
    df_ = pd.get_dummies(df.drop(cols_to_drop, axis=1))
    df_ = pca.fit_transform(scaler.fit_transform(df_))
    
    pca_comps = [f'pc_{n+1}' for n in range(df_.shape[1])]
    df_ = pd.DataFrame(df_, columns=pca_comps)
    df_['prob_V1_V2'] = df['prob_V1_V2']
    
    return df_

In [45]:
df_merged_pca = pca_transform(df_merged)

In [46]:
def knn_impute_prob(df_pca):
    mask = df_pca['prob_V1_V2'].notna()
    
    scaler = MinMaxScaler()

    X_pca_notna = scaler.fit_transform(df_pca.loc[mask].drop('prob_V1_V2', axis=1))
    y_pca_notna = df_pca.loc[mask, 'prob_V1_V2']

    X_pca_isna = scaler.transform(df_pca.loc[~mask].drop('prob_V1_V2', axis=1))
    
    knn_reg_imp = KNeighborsRegressor(weights='distance', n_jobs=-1)
    knn_reg_imp.fit(X_pca_notna, y_pca_notna)

    df_pca.loc[~mask, 'prob_V1_V2'] = knn_reg_imp.predict(X_pca_isna)
    
    return df_pca

In [84]:
df_merged_pca = knn_impute_prob(df_merged_pca)

In [99]:
# salvo as taxas imputadas em um novo .csv, contendo apenas as taxas

df_merged_pca['prob_V1_V2'].to_csv('../data/df_full_prob.csv', index=False)

In [101]:
# incluo as taxas imputados no dataframe mergeado anteriormente

df_merged['prob_V1_V2'] = df_merged_pca['prob_V1_V2']

In [305]:
# salvo o dataframe completo em um novo .csv

df_merged.to_csv('../data/df_full.csv', index=False)

In [4]:
df_merged = pd.read_csv('../data/df_full.csv')

In [5]:
df_merged.head()

,name_V1,idade_V1,qt_filhos_V1,estuda_V1,trabalha_V1,pratica_esportes_V1,IMC_V1,estado_civil-casado_V1,estado_civil-divorciado_V1,estado_civil-solteiro_V1,...,transporte_mais_utilizado-publico_V2,transporte_mais_utilizado-taxi_V2,prob_V1_V2,grau-amigos,grau-familia,grau-trabalho,proximidade-mora_junto,proximidade-visita_casual,proximidade-visita_frequente,proximidade-visita_rara
0,1,0.354839,0.111111,1.0,0.0,1.0,0.205777,0,1,0,...,1,0,0.589462,0,0,1,0,0,1,0
1,1,0.354839,0.111111,1.0,0.0,1.0,0.205777,0,1,0,...,0,0,0.708465,0,0,1,0,0,0,1
2,2,0.193548,0.000000,0.0,0.0,1.0,0.243471,1,0,0,...,1,0,0.650190,0,0,1,0,1,0,0
3,2,0.193548,0.000000,0.0,0.0,1.0,0.243471,1,0,0,...,1,0,0.638842,0,0,1,0,0,0,1
4,3,0.282258,0.111111,0.0,0.0,1.0,0.179106,0,0,1,...,1,0,0.300833,1,0,0,1,0,0,0


## Modelos de Regressão

In [6]:
# separação da variável target

x = pd.get_dummies(df_merged.drop(cols_to_drop, axis=1), drop_first=True)
y = df_merged['prob_V1_V2']

In [7]:
# separação dos dados em treino e teste

SEED = 26
np.random.seed(SEED)

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.25, random_state=42)

print(f"Treinaremos com {len(train_x)} elementos e testaremos com {len(test_x)} elementos")

Treinaremos com 749999 elementos e testaremos com 250000 elementos


In [8]:
# algoritmos e métrica R2 Score

models = [
#     svm.SVR(),
    linear_model.LinearRegression(),
    linear_model.SGDRegressor(),
    linear_model.BayesianRidge(),
    linear_model.ARDRegression(),
    linear_model.PassiveAggressiveRegressor()]
#     linear_model.TheilSenRegressor()]


for item in models:
    print(item)
    clf = item
    clf.fit(train_x, train_y)
    predict = clf.predict(test_x)
    print(clf.predict(test_x))
    score = r2_score(test_y, predict) * 100
    print("R2 Score %.2f%%" % score, '\n')

LinearRegression()
[0.40963745 0.61495972 0.64620972 ... 0.53567505 0.48544312 0.40164185]
R2 Score 56.22% 

SGDRegressor()
[0.41284796 0.61741154 0.64798692 ... 0.53683999 0.48640646 0.40753497]
R2 Score 56.18% 

BayesianRidge()
[0.40964899 0.61525323 0.64605932 ... 0.535865   0.48563896 0.40151494]
R2 Score 56.22% 

ARDRegression()
[0.41019121 0.61504349 0.64695975 ... 0.53729643 0.48594121 0.40222803]
R2 Score 56.22% 

PassiveAggressiveRegressor()
[0.36093608 0.54299805 0.58334576 ... 0.47315022 0.35277333 0.32273511]
R2 Score 23.17% 



Opto por fazer a validação cruzada com o algoritmo LinearRegression().

###  Validação cruzada dos dados

In [9]:
# cv = 3

SEED = 301
np.random.seed(SEED)

model = LinearRegression()

results = cross_validate(model, x, y, cv = 3, return_train_score=False)
mean = results['test_score'].mean()
std = results['test_score'].std()

print("Score com cross validation, 3 = [%.2f, %.2f]" % ((mean - 2 * std)*100,
                                                        (mean + 2 * std)*100))
print(results)

Score com cross validation, 3 = [55.86, 56.66]
{'fit_time': array([1.60659575, 1.44414306, 1.3702879 ]), 'score_time': array([0.10704207, 0.08200717, 0.04543996]), 'test_score': array([0.56533675, 0.56193811, 0.56055302])}


In [46]:
# cv = 10

SEED = 301
np.random.seed(SEED)

model = LinearRegression()

results = cross_validate(modelo, x, y, cv = 10, return_train_score=False)
mean = results['test_score'].mean()
std = results['test_score'].std()

print("Score com cross validation, 10 = [%.2f, %.2f]" % ((mean - 2 * std)*100,
                                                         (mean + 2 * std)*100))
print(results)

Score com cross validation, 10 = [55.62, 56.90]
{'fit_time': array([2.53510904, 2.05164599, 1.99804688, 1.98397279, 1.85266495,
       1.84970069, 1.86967111, 1.84067369, 1.86089396, 1.84738994]), 'score_time': array([0.02246881, 0.01467609, 0.01553512, 0.01438904, 0.01508021,
       0.01422524, 0.01399398, 0.0145731 , 0.01412511, 0.01471019]), 'test_score': array([0.56589613, 0.56638002, 0.56639624, 0.5586997 , 0.5640546 ,
       0.55883428, 0.56229165, 0.5650462 , 0.5586965 , 0.55964264])}


In [47]:
# cv = 5

SEED = 301
np.random.seed(SEED)

model = LinearRegression()

results = cross_validate(modelo, x, y, cv = 5, return_train_score=False)
mean = results['test_score'].mean()
std = results['test_score'].std()

print("Score com cross validation, 5 = [%.2f, %.2f]" % ((mean - 2 * std)*100, 
                                                           (mean + 2 * std)*100))
print(results)

Score com cross validation, 5 = [55.79, 56.72]
{'fit_time': array([2.00240517, 2.21604729, 1.71374011, 1.66063499, 1.65499997]), 'score_time': array([0.05433202, 0.0289259 , 0.02736902, 0.02679205, 0.02770114]), 'test_score': array([0.5661568 , 0.56255104, 0.56149217, 0.56363414, 0.55912805])}


### Aleatoriedade dos dados

In [48]:
def imprime_resultados(results):
    mean = results['test_score'].mean()
    std = results['test_score'].std()
    
    print("Score médio: %.2f" % (mean * 100))
    print("Score intervalo: [%.2f, %.2f]" % ((mean - 2 * std)*100, (mean + 2 * std) * 100))

In [49]:
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 5)

model = LinearRegression()
results = cross_validate(modelo, x, y, cv = cv, return_train_score=False)

imprime_resultados(results)

Score médio: 56.26
Score intervalo: [55.79, 56.72]


In [50]:
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle = True)

model = LinearRegression()
results = cross_validate(modelo, x, y, cv = cv, return_train_score=False)

imprime_resultados(results)

Score médio: 56.26
Score intervalo: [55.86, 56.66]


### Obersavções:

Mesmo optando por permanecer com a regressão linear, que não teve uma performance alta, com mais tempo disponível, além de ter a possibilidade de testar e avaliar outros algoritmos, poderiam ter sido testadas métricas diferentes do R2 Score. Também, poderiam ter sido realizados outros métodos de manipulação e tratamento dos dados, com imputações diferentes das taxas de contaminação, por exemplo. Pesquisando na literatura, algoritmos de redes neurais perfomam de maneira muito superior e mais satisfatória que as regressões, dado o pouco contato com redes neurais, optei por permanecer com a regressão.

## Completando os dados mergeados com a estimativas das taxas de contaminação

In [11]:
df_merged_full = df_merged.copy(deep=True)

In [14]:
re_cols = ['V2', 'V1', 'grau', 'proximidade']
re_con_df = df_conexoes[re_cols].copy(deep=True).rename(columns={'V1': 'V2', 'V2': 'V1'})
re_con_df['prob_V1_V2'] = np.nan

In [27]:
df_half = create_df(df_ind, df_con)

In [28]:
X_half = df_half.drop(cols_to_drop, axis=1)
X_half = pd.get_dummies(X_half, drop_first=True)

In [29]:
lr_model = LinearRegression().fit(train_x, train_y)

In [31]:
df_half['prob_V1_V2'] = lr_model.predict(X_half)

In [75]:
df = df_merged_full.append(df_half, ignore_index=True).drop(['name_V1', 'name_V2'], axis=1)

cols = ['idade_V1', 'qt_filhos_V1', 'estuda_V1', 'trabalha_V1', 'pratica_esportes_V1',
        'idade_V2', 'qt_filhos_V2', 'estuda_V2', 'trabalha_V2', 'pratica_esportes_V2']

df[cols] = df[cols].round()

## Revertendo a normalização e o encode dos dados

In [76]:
estado_civil = ['casado', 'divorciado', 'solteiro', 'viuvo']
transporte_mais_utilizado = ['particular', 'publico', 'taxi']
grau = ['amigos', 'familia', 'trabalho']
proximidade = ['mora_junto', 'visita_casual', 'visita_frequente', 'visita_rara']

In [77]:
def get_categorical(elements_list, categorical_values):
    values = []
    for element in elements_list:
        values.append(categorical_values[element.index(1.0)])
    return values

In [78]:
# estado civil

cols = ['estado_civil-casado_V1', 'estado_civil-divorciado_V1', 
        'estado_civil-solteiro_V1', 'estado_civil-viuvo_V1']
temp_data = df[cols].values.tolist()
df.drop(columns=cols, inplace=True)
col_data = get_categorical(temp_data, estado_civil)
df['estado_civil_V1'] = col_data

cols = ['estado_civil-casado_V2', 'estado_civil-divorciado_V2', 
        'estado_civil-solteiro_V2', 'estado_civil-viuvo_V2']
temp_data = df[cols].values.tolist()
df.drop(columns=cols, inplace=True)
col_data = get_categorical(temp_data, estado_civil)
df['estado_civil_V2'] = col_data

In [79]:
# transporte mais utilizado

cols = ['transporte_mais_utilizado-particular_V1', 
        'transporte_mais_utilizado-publico_V1', 
        'transporte_mais_utilizado-taxi_V1']
temp_data = df[cols].values.tolist()
df.drop(columns=cols, inplace=True)
col_data = get_categorical(temp_data, transporte_mais_utilizado)
df['transporte_mais_utilizado_V1'] = col_data

cols = ['transporte_mais_utilizado-particular_V2', 
        'transporte_mais_utilizado-publico_V2', 
        'transporte_mais_utilizado-taxi_V2']
temp_data = df[cols].values.tolist()
df.drop(columns=cols, inplace=True)
col_data = get_categorical(temp_data, transporte_mais_utilizado)
df['transporte_mais_utilizado_V2'] = col_data

In [80]:
# grau

cols = ['grau-amigos', 'grau-familia', 'grau-trabalho']
temp_data = df[cols].values.tolist()
df.drop(columns=cols, inplace=True)
col_data = get_categorical(temp_data, grau)
df['grau'] = col_data

In [81]:
# proximidade

cols = ['proximidade-mora_junto', 'proximidade-visita_casual', 
        'proximidade-visita_frequente', 'proximidade-visita_rara']
temp_data = df[cols].values.tolist()
df.drop(columns=cols, inplace=True)
col_data = get_categorical(temp_data, proximidade)
df['proximidade'] = col_data

In [79]:
# revertendo a normalização

def denormalize(df, col, norm_params):
    maxim = norm_params[f'max_{col}']
    minim = norm_params[f'min_{col}']
    v_col = f'{col}_V1'
    df[v_col] = (df[v_col]*(maxim - minim)) + minim
    v_col = f'{col}_V2'
    df[v_col] = (df[v_col]*(maxim - minim)) + minim
cols = ['idade', 'qt_filhos', 'IMC']
for col in tqdm(cols):
    denormalize(df, col, norm_params)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  3.55it/s]


In [99]:
# salvando o dataframe final, com as estimativas completas, em um novo .csv

df.to_csv('../data/df_full_denorm.csv', index=False)

In [100]:
df.head()

,idade_V1,qt_filhos_V1,estuda_V1,trabalha_V1,pratica_esportes_V1,IMC_V1,idade_V2,qt_filhos_V2,estuda_V2,trabalha_V2,pratica_esportes_V2,IMC_V2,prob_V1_V2,estado_civil_V1,estado_civil_V2,transporte_mais_utilizado_V1,transporte_mais_utilizado_V2,grau,proximidade
0,44.0,1.0,1.0,0.0,1.0,22.200956,24.0,0.0,0.0,0.0,1.0,25.378720,0.589462,divorciado,casado,publico,publico,trabalho,visita_frequente
1,44.0,1.0,1.0,0.0,1.0,22.200956,35.0,1.0,0.0,0.0,1.0,19.952393,0.708465,divorciado,solteiro,publico,particular,trabalho,visita_rara
2,24.0,0.0,0.0,0.0,1.0,25.378720,50.0,1.0,1.0,1.0,0.0,26.732053,0.602863,casado,casado,publico,publico,trabalho,visita_casual
3,24.0,0.0,0.0,0.0,1.0,25.378720,30.0,2.0,1.0,0.0,1.0,15.295668,0.638842,casado,solteiro,publico,publico,trabalho,visita_rara
4,35.0,1.0,0.0,0.0,1.0,19.952393,20.0,1.0,0.0,1.0,0.0,20.412942,0.232270,solteiro,solteiro,particular,publico,amigos,mora_junto


### Principais referências:

- [Documentação scikit-learn](https://scikit-learn.org/stable/)
- [Hands-On Machine Learning with Scikit-Learn and TensorFlow](https://www.oreilly.com/library/view/hands-on-machine-learning/9781491962282/)